# TeleChurn Predictor: Exploratory Data Analysis

This notebook performs initial exploratory data analysis on telecom customer data for the TeleChurn Predictor project. The goal is to understand the data structure, identify patterns, and gain insights that will inform feature engineering and model development.

## Project Overview

TeleChurn Predictor is a machine learning system designed to predict customer churn in the telecommunications industry. This notebook is the first step in the data science pipeline, focusing on understanding the raw data before preprocessing and modeling.

## Contents

1. [Data Loading and Initial Inspection](#1.-Data-Loading-and-Initial-Inspection)
2. [Summary Statistics and Data Types](#2.-Summary-Statistics-and-Data-Types)
3. [Missing Value Analysis](#3.-Missing-Value-Analysis)
4. [Distribution of Numerical Features](#4.-Distribution-of-Numerical-Features)
5. [Analysis of Categorical Features](#5.-Analysis-of-Categorical-Features)
6. [Target Variable Analysis](#6.-Target-Variable-Analysis)
7. [Correlation Analysis](#7.-Correlation-Analysis)
8. [Outlier Detection](#8.-Outlier-Detection)
9. [Feature Importance Analysis](#9.-Feature-Importance-Analysis)
10. [Conclusions and Next Steps](#10.-Conclusions-and-Next-Steps)

## Setup

Let's import the necessary libraries and set up the environment.

In [ ]:
# Install required libraries (if not already installed)
!pip install pandas numpy matplotlib seaborn scipy missingno

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import missingno as msno
import warnings

# Configure visualizations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette('viridis')

# Ignore warnings
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

## 1. Data Loading and Initial Inspection

Let's load the telecom customer data and perform an initial inspection.

In [ ]:
# Define file paths
train_path = "../data/raw/cell2celltrain.csv"
holdout_path = "../data/raw/cell2cellholdout.csv"

# Load data
train_data = pd.read_csv(train_path)
holdout_data = pd.read_csv(holdout_path)

print(f"Training data shape: {train_data.shape}")
print(f"Holdout data shape: {holdout_data.shape}")

In [ ]:
# Display the first few rows of the training data
train_data.head()

In [ ]:
# Display the first few rows of the holdout data
holdout_data.head()

### Check for duplicates

In [ ]:
# Check for duplicate rows in training data
print(f"Number of duplicate rows in training data: {train_data.duplicated().sum()}")

# Check for duplicate CustomerIDs in training data
print(f"Number of duplicate CustomerIDs in training data: {train_data['CustomerID'].duplicated().sum()}")

## 2. Summary Statistics and Data Types

Let's examine the data types and summary statistics of our dataset.

In [ ]:
# Display data types
train_data.dtypes.sort_values()

In [ ]:
# Summary statistics for numerical columns
train_data.describe().T.sort_values(by='mean', ascending=False)

In [ ]:
# Summary statistics for categorical columns
categorical_columns = train_data.select_dtypes(include=['object']).columns

for column in categorical_columns:
    print(f"\n{column}:")
    print(train_data[column].value_counts(normalize=True).head(10) * 100)

### Categorize features

Let's categorize our features into different types for easier analysis.

In [ ]:
# Categorize features
id_columns = ['CustomerID']
target_column = ['Churn']

# Categorical columns (non-binary, non-numeric)
categorical_columns = [
    'ServiceArea', 'CreditRating', 'PrizmCode', 'Occupation', 'MaritalStatus'
]

# Binary columns (yes/no or similar)
binary_columns = [
    'ChildrenInHH', 'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner',
    'RVOwner', 'Homeownership', 'BuysViaMailOrder', 'RespondsToMailOffers',
    'OptOutMailings', 'NonUSTravel', 'OwnsComputer', 'HasCreditCard',
    'NewCellphoneUser', 'NotNewCellphoneUser', 'OwnsMotorcycle',
    'MadeCallToRetentionTeam'
]

# Numerical columns (continuous)
numerical_continuous_columns = [
    'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
    'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
    'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
    'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls',
    'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
    'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls',
    'CallForwardingCalls', 'CallWaitingCalls', 'AgeHH1', 'AgeHH2'
]

# Numerical columns (discrete/integer)
numerical_discrete_columns = [
    'MonthsInService', 'UniqueSubs', 'ActiveSubs', 'Handsets',
    'HandsetModels', 'CurrentEquipmentDays', 'RetentionCalls',
    'RetentionOffersAccepted', 'ReferralsMadeBySubscriber',
    'IncomeGroup', 'AdjustmentsToCreditRating', 'HandsetPrice'
]

# All numerical columns
numerical_columns = numerical_continuous_columns + numerical_discrete_columns

# Print counts
print(f"Total features: {len(train_data.columns)}")
print(f"ID columns: {len(id_columns)}")
print(f"Target column: {len(target_column)}")
print(f"Categorical columns: {len(categorical_columns)}")
print(f"Binary columns: {len(binary_columns)}")
print(f"Numerical continuous columns: {len(numerical_continuous_columns)}")
print(f"Numerical discrete columns: {len(numerical_discrete_columns)}")

## 3. Missing Value Analysis

Let's analyze missing values in our dataset.

In [ ]:
# Calculate missing values
missing = train_data.isnull().sum()
missing_percent = missing / len(train_data) * 100

# Create summary DataFrame
missing_summary = pd.DataFrame({
    'Missing Values': missing,
    'Missing Percentage': missing_percent
})

# Sort by missing percentage
missing_summary = missing_summary.sort_values('Missing Percentage', ascending=False)

# Display columns with missing values
missing_summary[missing_summary['Missing Values'] > 0]

In [ ]:
# Visualize missing value correlations
plt.figure(figsize=(12, 10))
msno.heatmap(train_data)
plt.title('Missing Value Correlation Heatmap')
plt.show()

## 4. Distribution of Numerical Features

Let's examine the distribution of numerical features.

In [ ]:
# Function to plot histograms for numerical features
def plot_histograms(data, columns, bins=30, figsize=(16, 12), ncols=3):
    nrows = int(np.ceil(len(columns) / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()
    
    for i, column in enumerate(columns):
        if i < len(axes):
            sns.histplot(data[column], bins=bins, kde=True, ax=axes[i])
            axes[i].set_title(f'Distribution of {column}')
            axes[i].set_xlabel(column)
            axes[i].set_ylabel('Frequency')
    
    # Hide unused subplots
    for j in range(len(columns), len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot histograms for key numerical continuous features
key_continuous_features = [
    'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
    'OverageMinutes', 'PercChangeMinutes', 'PercChangeRevenues',
    'DroppedCalls', 'BlockedCalls', 'UnansweredCalls'
]

plot_histograms(train_data, key_continuous_features)

In [ ]:
# Plot histograms for key numerical discrete features
key_discrete_features = [
    'MonthsInService', 'UniqueSubs', 'ActiveSubs', 'Handsets',
    'HandsetModels', 'CurrentEquipmentDays', 'RetentionCalls',
    'IncomeGroup', 'HandsetPrice'
]

plot_histograms(train_data, key_discrete_features)

### Box plots for numerical features

Let's use box plots to identify potential outliers in our numerical features.

In [ ]:
# Function to plot box plots for numerical features
def plot_boxplots(data, columns, figsize=(16, 12), ncols=3):
    nrows = int(np.ceil(len(columns) / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()
    
    for i, column in enumerate(columns):
        if i < len(axes):
            sns.boxplot(x=data[column], ax=axes[i])
            axes[i].set_title(f'Box Plot of {column}')
            axes[i].set_xlabel(column)
    
    # Hide unused subplots
    for j in range(len(columns), len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot box plots for key numerical continuous features
plot_boxplots(train_data, key_continuous_features)

## 5. Analysis of Categorical Features

Let's examine the distribution of categorical features.

In [ ]:
# Function to plot bar charts for categorical features
def plot_bar_charts(data, columns, figsize=(16, 12), ncols=2):
    nrows = int(np.ceil(len(columns) / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()
    
    for i, column in enumerate(columns):
        if i < len(axes):
            value_counts = data[column].value_counts().sort_values(ascending=False)
            sns.barplot(x=value_counts.index, y=value_counts.values, ax=axes[i])
            axes[i].set_title(f'Distribution of {column}')
            axes[i].set_xlabel(column)
            axes[i].set_ylabel('Count')
            axes[i].tick_params(axis='x', rotation=45)
    
    # Hide unused subplots
    for j in range(len(columns), len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot bar charts for categorical features
plot_bar_charts(train_data, categorical_columns)

In [ ]:
# Plot bar charts for binary features
plot_bar_charts(train_data, binary_columns, ncols=3)

## 6. Target Variable Analysis

Let's analyze the target variable (Churn) and its relationship with other features.

In [ ]:
# Distribution of the target variable
churn_counts = train_data['Churn'].value_counts()
churn_percent = train_data['Churn'].value_counts(normalize=True) * 100

plt.figure(figsize=(10, 6))
ax = sns.barplot(x=churn_counts.index, y=churn_counts.values)
plt.title('Distribution of Churn')
plt.xlabel('Churn')
plt.ylabel('Count')

# Add percentage labels
for i, p in enumerate(ax.patches):
    ax.annotate(f"{churn_percent.values[i]:.1f}%", 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=12)

plt.show()

### Relationship between numerical features and churn

In [ ]:
# Function to plot numerical features by churn
def plot_numerical_by_churn(data, columns, figsize=(16, 12), ncols=3):
    nrows = int(np.ceil(len(columns) / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()
    
    for i, column in enumerate(columns):
        if i < len(axes):
            sns.boxplot(x='Churn', y=column, data=data, ax=axes[i])
            axes[i].set_title(f'{column} by Churn')
            axes[i].set_xlabel('Churn')
            axes[i].set_ylabel(column)
    
    # Hide unused subplots
    for j in range(len(columns), len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot key numerical features by churn
key_features_for_churn = [
    'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
    'OverageMinutes', 'PercChangeMinutes', 'PercChangeRevenues',
    'DroppedCalls', 'BlockedCalls', 'UnansweredCalls',
    'CustomerCareCalls', 'MonthsInService', 'RetentionCalls'
]

plot_numerical_by_churn(train_data, key_features_for_churn)

### Relationship between categorical features and churn

In [ ]:
# Function to plot categorical features by churn
def plot_categorical_by_churn(data, columns, figsize=(16, 12), ncols=2):
    nrows = int(np.ceil(len(columns) / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()
    
    for i, column in enumerate(columns):
        if i < len(axes):
            # Calculate churn rate by category
            churn_rate = data.groupby(column)['Churn'].apply(lambda x: (x == 'Yes').mean() * 100).sort_values(ascending=False)
            
            # Plot
            sns.barplot(x=churn_rate.index, y=churn_rate.values, ax=axes[i])
            axes[i].set_title(f'Churn Rate by {column}')
            axes[i].set_xlabel(column)
            axes[i].set_ylabel('Churn Rate (%)')
            axes[i].tick_params(axis='x', rotation=45)
            
            # Add count labels
            for j, p in enumerate(axes[i].patches):
                category = churn_rate.index[j]
                count = len(data[data[column] == category])
                axes[i].annotate(f"n={count}", 
                            (p.get_x() + p.get_width() / 2., p.get_height() + 1), 
                            ha='center', va='bottom', fontsize=9)
    
    # Hide unused subplots
    for j in range(len(columns), len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot categorical features by churn
plot_categorical_by_churn(train_data, categorical_columns)

In [ ]:
# Plot binary features by churn
plot_categorical_by_churn(train_data, binary_columns, ncols=3)

## 7. Correlation Analysis

Let's analyze correlations between numerical features and with the target variable.

In [ ]:
# Create a copy of the data with binary target
train_data_corr = train_data.copy()
train_data_corr['Churn_Binary'] = train_data_corr['Churn'].map({'Yes': 1, 'No': 0})

# Make sure we only include numeric columns
numeric_cols = train_data_corr.select_dtypes(include=['number']).columns
corr_data = train_data_corr[numeric_cols].copy()

# Check for any remaining non-numeric values
print("Checking for non-numeric values in correlation data:")
for col in corr_data.columns:
    non_numeric = corr_data[col].map(lambda x: not np.issubdtype(type(x), np.number)).sum()
    if non_numeric > 0:
        print(f"Column {col} has {non_numeric} non-numeric values")
        # Convert to numeric, coercing errors to NaN
        corr_data[col] = pd.to_numeric(corr_data[col], errors='coerce')

In [ ]:
# Calculate correlation matrix
corr_matrix = corr_data.corr()

# Plot correlation heatmap
plt.figure(figsize=(16, 14))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='coolwarm', center=0, linewidths=0.5)
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()

In [ ]:
# Top correlations with Churn
if 'Churn_Binary' in corr_matrix.columns:
    churn_corr = corr_matrix['Churn_Binary'].sort_values(ascending=False)
    print("Top positive correlations with Churn:")
    print(churn_corr.head(10))
    print("\nTop negative correlations with Churn:")
    print(churn_corr.tail(10))

In [ ]:
# Plot top correlations with Churn
if 'Churn_Binary' in corr_matrix.columns:
    plt.figure(figsize=(12, 8))
    top_corr = pd.concat([churn_corr.head(10), churn_corr.tail(10)])
    top_corr = top_corr.drop('Churn_Binary')  # Remove self-correlation
    sns.barplot(x=top_corr.values, y=top_corr.index)
    plt.title('Top Correlations with Churn')
    plt.xlabel('Correlation Coefficient')
    plt.axvline(x=0, color='black', linestyle='--')
    plt.tight_layout()
    plt.show()

### Highly correlated features

Let's identify pairs of highly correlated features.

In [ ]:
# Function to get highly correlated pairs
def get_highly_correlated_pairs(corr_matrix, threshold=0.7):
    # Create a mask for the upper triangle
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    
    # Apply mask and get pairs with correlation above threshold
    corr_pairs = corr_matrix.mask(mask).stack().reset_index()
    corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
    
    # Filter by threshold and sort
    high_corr_pairs = corr_pairs[abs(corr_pairs['Correlation']) > threshold]
    high_corr_pairs = high_corr_pairs.sort_values('Correlation', ascending=False)
    
    return high_corr_pairs

# Get highly correlated pairs
if 'Churn_Binary' in corr_matrix.columns:
    high_corr_pairs = get_highly_correlated_pairs(corr_matrix.drop('Churn_Binary', axis=1).drop('Churn_Binary', axis=0))
else:
    high_corr_pairs = get_highly_correlated_pairs(corr_matrix)
    
high_corr_pairs

## 8. Outlier Detection

Let's identify outliers in our numerical features.

In [ ]:
# Function to detect outliers using IQR method
def detect_outliers_iqr(data, columns):
    outlier_counts = {}
    
    for column in columns:
        if column in data.columns and pd.api.types.is_numeric_dtype(data[column]):
            Q1 = data[column].quantile(0.25)
            Q3 = data[column].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
            outlier_counts[column] = len(outliers)
    
    return pd.Series(outlier_counts).sort_values(ascending=False)

# Detect outliers in numerical features
outlier_counts = detect_outliers_iqr(train_data, numerical_columns)
outlier_percent = outlier_counts / len(train_data) * 100

# Create summary DataFrame
outlier_summary = pd.DataFrame({
    'Outlier Count': outlier_counts,
    'Outlier Percentage': outlier_percent
})

# Display features with outliers
outlier_summary[outlier_summary['Outlier Count'] > 0].sort_values('Outlier Percentage', ascending=False).head(20)

In [ ]:
# Plot outlier percentages
plt.figure(figsize=(12, 8))
top_outliers = outlier_summary[outlier_summary['Outlier Count'] > 0].sort_values('Outlier Percentage', ascending=False).head(15)
sns.barplot(x=top_outliers['Outlier Percentage'], y=top_outliers.index)
plt.title('Features with Highest Percentage of Outliers')
plt.xlabel('Outlier Percentage (%)')
plt.tight_layout()
plt.show()

## 9. Feature Importance Analysis

Let's perform a preliminary feature importance analysis using a simple model.

In [ ]:
# Prepare data for modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Create a copy of the data
model_data = train_data.copy()

# Encode target variable
model_data['Churn'] = model_data['Churn'].map({'Yes': 1, 'No': 0})

# Check for non-numeric values in numerical columns
for column in numerical_columns:
    if column in model_data.columns:
        # Convert to numeric, coercing errors to NaN
        model_data[column] = pd.to_numeric(model_data[column], errors='coerce')

# Handle missing values in numerical features
numerical_imputer = SimpleImputer(strategy='median')
model_data[numerical_columns] = numerical_imputer.fit_transform(model_data[numerical_columns])

# Encode categorical features
for column in categorical_columns + binary_columns:
    if column in model_data.columns:
        le = LabelEncoder()
        model_data[column] = le.fit_transform(model_data[column].astype(str))

# Scale numerical features
scaler = StandardScaler()
model_data[numerical_columns] = scaler.fit_transform(model_data[numerical_columns])

# Prepare features and target
features = model_data.drop(['CustomerID', 'Churn'], axis=1)
target = model_data['Churn']

In [ ]:
# Train a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(features, target)

# Get feature importances
feature_importances = pd.DataFrame({
    'Feature': features.columns,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

# Display top features
feature_importances.head(20)

In [ ]:
# Plot feature importances
plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(20))
plt.title('Top 20 Feature Importances')
plt.tight_layout()
plt.show()

## 10. Conclusions and Next Steps

### Key Findings

1. **Target Variable Distribution:**
   - The dataset has an imbalanced distribution of the target variable (Churn).
   - [Add specific percentages based on actual data]

2. **Missing Values:**
   - [Summarize missing value findings]
   - [Mention any patterns in missing data]

3. **Feature Distributions:**
   - Many numerical features have skewed distributions, suggesting the need for transformations.
   - Several features contain outliers that may need special handling.

4. **Correlations:**
   - [Highlight key correlations with churn]
   - [Mention highly correlated feature pairs that might cause multicollinearity]

5. **Feature Importance:**
   - [List top features identified by the Random Forest model]
   - These features should be given special attention in the feature engineering phase.

### Next Steps

1. **Data Preprocessing:**
   - Handle missing values using appropriate imputation techniques.
   - Address outliers through capping, removal, or transformation.
   - Apply feature scaling to normalize numerical features.

2. **Feature Engineering:**
   - Create interaction features between key variables.
   - Develop ratio features (e.g., revenue per minute).
   - Generate aggregated features for call patterns.
   - Consider dimensionality reduction for highly correlated features.

3. **Model Development:**
   - Implement multiple models (Random Forest, XGBoost, Neural Networks).
   - Address class imbalance through sampling techniques or class weights.
   - Perform hyperparameter optimization.
   - Evaluate models using appropriate metrics (AUC, F1-score, precision, recall).

4. **Model Interpretation:**
   - Use SHAP values to explain model predictions.
   - Identify key factors driving churn.
   - Develop actionable insights for business stakeholders.

The next notebook in this series will focus on feature engineering based on the insights gained from this exploratory analysis.